40 points task. 2 hour task

You have only one pass attempt

* Load CIFAR10. And keep only 1000 samples per each class from train (for speedup). Drop test data. All experiments you have to perform on train data (using cross-validation) and report only validation accuracy
    * from keras.datasets import cifar10
    * (x_train, y_train), (x_test, y_test) = cifar10.load_data()
* Train Random Forest on original data with light hyperparameters search
* Train CNN on original data
* Train Random Forest on latent vector (output of fully-connected layer) of trained CNN. With light hyperparameters search
* Compare results of trained models

TIPS:
If you have any errors with tf or keras try: tensorflow==1.8.0, keras==2.2.0


In [2]:
from __future__ import print_function
from keras.datasets import cifar10
import keras
from keras.layers.core import Dense, Dropout
from keras.layers import Conv2D, MaxPool2D, Flatten, UpSampling2D, MaxPooling2D
from keras.layers import Input
from keras.utils import np_utils
from keras.models import Model
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.callbacks import TensorBoard
import matplotlib.pyplot as plt
from sklearn.model_selection import ParameterGrid
from keras.regularizers import l2,l1
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold



os.environ['KMP_DUPLICATE_LIB_OK']='True'

## Load Data

In [4]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 107s 1us/step


In [ ]:
rf = RandomForestClassifier()
rf.fit()